
# Оптимизация гиперпараметров

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Задания для выполнения:

### 1. Сгенерируйте данные для задачи бинарной классификации с 4 признаками и 10000 наблюдениями. Целевая переменная должна зависеть от 2 признаков. Выведите признаки в виде датафрейма, дав названия колонкам. Выведите число объектов в каждом классе.
Указание: Для этого воспользуйтесь [make_classification](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html), задав параметры: число строк n_samples=10000, число признаков n_features=4, число признаков от которых зависит целевая переменная n_informative=2, число избыточных признаков (т.е. признаков, являющихся линейными комбинациями признаков от которых зависит целевая переменная) n_redundant=2, фиксируем воспроизводимость случайных данных random_state=42.

In [ ]:
X, y = datasets.make_classification(n_samples=10000, n_features=4, n_informative=2, n_redundant=2, random_state=42)

In [ ]:
X[:5]

array([[ 1.52264971, -0.93455988, -0.46502165,  0.05887383],
       [ 1.04810323, -0.74680553,  0.43685274,  0.85962787],
       [ 1.02421601, -0.44941167, -1.62343415, -1.37865568],
       [-0.43466749,  0.2817535 ,  0.02329917, -0.13524348],
       [ 1.42116496, -0.73157419, -1.46292601, -1.05843534]])

In [ ]:
df=pd.DataFrame(data=X, columns =['X'+ str(i) for i in range(1, X.shape[1]+1)])
df.head()

,X1,X2,X3,X4
0,1.522650,-0.934560,-0.465022,0.058874
1,1.048103,-0.746806,0.436853,0.859628
2,1.024216,-0.449412,-1.623434,-1.378656
3,-0.434667,0.281754,0.023299,-0.135243
4,1.421165,-0.731574,-1.462926,-1.058435


In [ ]:
pd.Series(y).value_counts()

,count
0,5000
1,5000


### 2. Разделите данные на обучающую и тестовую части. В тестовую часть отправьте 20% объектов и зафиксируйте способ перемешивания данных random_state=42. Выведите размеры обучающей и тестовой частей.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape[0], X_test.shape[0]

(8000, 2000)

### 3. Постройте модель логистической регрессии с гиперпараметрами, установленными по умолчанию. Для этого создайте класс LogisticRegression c дефолтными гиперпараметрами и обучите модель на обучающих данных. Сделайте предсказание на тестовых данных. Вычислите метрику accuracy на тестовых данных.

In [ ]:
model_defolt = LogisticRegression()
model_defolt.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [ ]:
model_defolt.fit(X_train, y_train)
accuracy_defolt = model_defolt.score(X_test, y_test)
accuracy_defolt

0.8935

In [ ]:
#другой способ
y_test_pred = model_defolt.predict(X_test)
accuracy_defolt=metrics.accuracy_score(y_test, y_test_pred)
accuracy_defolt

0.8935

### 4. Напишите функцию, принимающую матрицу признаков и вектор целей и осуществляющую решетчатый поиск лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием hold-out разбиения, среди двух гиперпараметров, значения которых указанны ниже. Подберите лучшие гиперпараметры на обучающих данных. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрику accuracy на тестовых данных.

Создадим диапазон вариантов для гиперпараметров в виде словаря:

In [ ]:
param_grid = {'C': np.logspace(-3,2,9),
              'solver': ['lbfgs','liblinear', 'saga']
}

In [ ]:
def Grid(X,y):
    X_trainval, X_testval, y_trainval, y_testval = train_test_split(X, y, test_size=0.2, random_state=42)
    score=0
    best_score=0
    for C in param_grid['C']:
      for solver in param_grid['solver']:
        model=LogisticRegression(C=C, solver=solver)
        model.fit(X_trainval,y_trainval)
        score = model.score(X_testval, y_testval)
        if score > best_score:
          best_score= score
          best_parameters= {'C':C, 'solver':solver}

    return  best_parameters

In [ ]:
best_parameters = Grid(X_train, y_train)
best_parameters

{'C': 0.004216965034285823, 'solver': 'lbfgs'}

In [ ]:
best_model = LogisticRegression(**best_parameters)
best_model.fit(X_train, y_train)
accyracy_best= best_model.score(X_test,y_test)
accyracy_best, accuracy_defolt

(0.894, 0.8935)

### 5. Напишите функцию, принимающую матрицу признаков и вектор целей и осуществляющую решетчатый поиск лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, для тех же гиперпараметров, что и в предыдущем задании. Подберите лучшие гиперпараметры на обучающих данных. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрику accuracy на тестовых данных.

In [ ]:
def GridCV(X,y):
    score=0
    best_score=0
    for C in param_grid['C']:
      for solver in param_grid['solver']:
        model=LogisticRegression(C=C, solver=solver)
        score = np.mean(cross_val_score(model, X, y, cv = 5, scoring = 'accuracy'))
        if score > best_score:
          best_score= score
          best_parameters= {'C':C, 'solver':solver}

    return  best_parameters

In [ ]:
best_parameters  = GridCV(X_train, y_train)
best_parameters

{'C': 0.01778279410038923, 'solver': 'saga'}

### 6. Осуществите решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, для тех же гиперпараметров, что и в предыдущем задании, используя класс GridSearchCV из библиотеки sklearn. Убедитесь, что результаты будут теми же, что и в предыдущем задании.
Указание: Для выведения наилучших гиперпараметров модели, используйте атрибут best_params_. Наилучшая модель выводится при помощи атрибута best_estimator_.

In [ ]:
gs= GridSearchCV(
    estimator=LogisticRegression(),
    param_grid= param_grid,
    cv=5,
    scoring='accuracy'
)

In [ ]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-03, 4.21696503e-03, 1.77827941e-02, 7.49894209e-02,
       3.16227766e-01, 1.33352143e+00, 5.62341325e+00, 2.37137371e+01,
       1.00000000e+02]),
                         'solver': ['lbfgs', 'liblinear', 'saga']},
             scoring='accuracy')

In [ ]:
gs.best_params_

{'C': 0.01778279410038923, 'solver': 'saga'}

In [ ]:
best_model = LogisticRegression(**gs.best_params_)
best_model.fit(X_train, y_train)
accyracy_best= best_model.score(X_test,y_test)
accyracy_best, accuracy_defolt

(0.894, 0.8935)

### 7. Выведите результаты решетчатого поиска с использованием класса GridSearchCV, осуществлённого в предыдущем пункте, в виде словаря и в виде датафрейма. Оставьте в датафрейме только два столбца со значениями гиперпараметров и со значениями вычисляемой метрики. Выведите самое большое значение среди средних значений метрики accuracy перекрестной проверки.
Указание: Для выведения результатов решетчатого поиска в виде словаря используйте атрибут cv_results_. Для выведения самого большого значения среди средних значений метрики accuracy используйте атрибут best_score_.

In [ ]:
gs.cv_results_

{'mean_fit_time': array([0.02391372, 0.01442513, 0.02200828, 0.01796017, 0.01867228,
        0.01699281, 0.00715084, 0.00854578, 0.01789989, 0.00830293,
        0.00836201, 0.01751318, 0.00842428, 0.00912595, 0.01646461,
        0.0085072 , 0.00906167, 0.01709876, 0.0136044 , 0.00937209,
        0.0165803 , 0.00859981, 0.00932102, 0.01794243, 0.00915184,
        0.00931115, 0.01794419]),
 'std_fit_time': array([3.57414094e-03, 1.05039835e-03, 3.81635542e-03, 1.24183391e-03,
        1.86513099e-03, 4.86505038e-03, 3.68838008e-04, 1.09418139e-04,
        4.11826204e-03, 2.12859862e-04, 4.78914979e-05, 3.64067743e-03,
        3.28383491e-04, 2.56413435e-04, 5.45076083e-03, 4.76337658e-04,
        3.93290206e-04, 5.86273627e-03, 6.45868384e-03, 1.19405452e-04,
        4.78141810e-03, 5.39138826e-04, 7.69852669e-05, 3.84918737e-03,
        1.31597652e-03, 2.18809365e-04, 3.46492439e-03]),
 'mean_score_time': array([0.0025291 , 0.00519452, 0.00185599, 0.00201716, 0.00377736,
        0.001504

In [ ]:
pd.DataFrame(gs.cv_results_).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.023914,0.003574,0.002529,0.001261,0.001000,lbfgs,"{'C': 0.001, 'solver': 'lbfgs'}",0.88125,0.878125,0.886875,0.894375,0.895000,0.887125,0.006785,26
1,0.014425,0.001050,0.005195,0.001905,0.001000,liblinear,"{'C': 0.001, 'solver': 'liblinear'}",0.88375,0.878750,0.887500,0.891875,0.895000,0.887375,0.005761,25
2,0.022008,0.003816,0.001856,0.000052,0.001000,saga,"{'C': 0.001, 'solver': 'saga'}",0.88125,0.878125,0.886875,0.894375,0.895000,0.887125,0.006785,26
3,0.017960,0.001242,0.002017,0.000621,0.004217,lbfgs,"{'C': 0.004216965034285823, 'solver': 'lbfgs'}",0.88625,0.881875,0.891875,0.900000,0.896875,0.891375,0.006654,8
4,0.018672,0.001865,0.003777,0.002428,0.004217,liblinear,"{'C': 0.004216965034285823, 'solver': 'libline...",0.88750,0.883125,0.891875,0.900625,0.896250,0.891875,0.006187,4


In [ ]:
pd.DataFrame({'params' : gs.cv_results_['params'], 'mean_test_score':gs.cv_results_['mean_test_score']})

,params,mean_test_score
0,"{'C': 0.001, 'solver': 'lbfgs'}",0.887125
1,"{'C': 0.001, 'solver': 'liblinear'}",0.887375
2,"{'C': 0.001, 'solver': 'saga'}",0.887125
3,"{'C': 0.004216965034285823, 'solver': 'lbfgs'}",0.891375
4,"{'C': 0.004216965034285823, 'solver': 'libline...",0.891875
5,"{'C': 0.004216965034285823, 'solver': 'saga'}",0.891375
6,"{'C': 0.01778279410038923, 'solver': 'lbfgs'}",0.892375
7,"{'C': 0.01778279410038923, 'solver': 'liblinear'}",0.892250
8,"{'C': 0.01778279410038923, 'solver': 'saga'}",0.892500
9,"{'C': 0.07498942093324558, 'solver': 'lbfgs'}",0.891625


In [ ]:
gs.best_score_.round(6)

0.8925

In [ ]:
gs.cv_results_['mean_test_score'].max().round(6)

0.8925

### 8. Используйте методы score и predict класса GridSearchCV, чтобы вычислить метрики качества accuracy и F1-меры на тестовых данных для модели LogisticRegression с лучшими гиперпараметрами.

In [ ]:
gs.score(X_test, y_test)

0.894

In [ ]:
y_test_pred = gs.predict(X_test)
metrics.f1_score(y_test, y_test_pred)

0.8922764227642277

### 9. Осуществите решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, используя класс GridSearchCV из библиотеки sklearn, среди списка словарей гиперпараметров, указанных ниже. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрики accuracy и F1-меру на тестовых данных.

In [ ]:
param_grid = param_grid = [{'solver': ['liblinear'],
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-3,2,9),
    'max_iter' : [1000]
   }, {
    'solver': ['saga'],
    'penalty': ['elasticnet'],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
    'max_iter' : [1000]
}]

In [ ]:
gs_list= GridSearchCV(
    estimator=LogisticRegression(),
    param_grid= param_grid,
    cv=5,
    scoring='accuracy'
)

In [ ]:
gs_list.fit(X_train, y_train)
gs_list.best_params_

{'C': 0.01778279410038923,
 'max_iter': 1000,
 'penalty': 'l1',
 'solver': 'liblinear'}

In [ ]:
acc = gs_list.score(X_test, y_test)
acc

0.894

In [ ]:
y_test_pred = gs_list.predict(X_test)
f1 = metrics.f1_score(y_test, y_test_pred)
f1

0.8928210313447927

### 10. Осуществите рандомизированный решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, используя класс RandomizedSearchCV из библиотеки sklearn, для списка словарей гиперпараметров из предыдущего задания. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрики accuracy и F1-меру на тестовых данных.

In [ ]:
rgs= RandomizedSearchCV(
    estimator=LogisticRegression(),
    param_distributions= param_grid,
    cv=5,
    scoring='accuracy',
    random_state=1
)

In [ ]:
rgs.fit(X_train, y_train)
rgs.best_params_

{'solver': 'liblinear',
 'penalty': 'l1',
 'max_iter': 1000,
 'C': 0.01778279410038923}

In [ ]:
pd.DataFrame({'params' : rgs.cv_results_['params'], 'mean_test_score':rgs.cv_results_['mean_test_score']})

,params,mean_test_score
0,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.890750
1,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.890875
2,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.891875
3,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.890875
4,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.890750
5,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.890875
6,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.890750
7,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.893250
8,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.888625
9,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.890875


# Задания для самостоятельного выполнения:

### 1. Загрузите встроенный датасет `load_iris`. Обозначьте признаки за `X`, а целевую переменную за `y`. Выведите первые 5 строк в виде датафрейма.

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
iris = load_iris()

In [ ]:
X = iris.data
y = iris.target

In [ ]:
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

In [ ]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### 2. Разделите данные на обучающую и тестовую части. В тестовую часть отправьте 15% объектов и зафиксируйте способ перемешивания данных random_state=0. Выведите размеры обучающей и тестовой частей.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

In [ ]:
X_train.shape[0], X_test.shape[0]

(127, 23)

### 3. Для датасета load_iris осуществите классификацию методом логистической регрессии, создав класс LogisticRegression(max_iter = 1000). Выведите значения гиперпараметров этой модели, установленные по умолчанию. Обучите эту модель, сделайте предсказание. Оцените качество модели, выведя значения метрик accuracy и f1-score.

In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 1000,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [ ]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
accuracy = model.score(X_test, y_test)
accuracy

1.0

In [ ]:
y_test_pred = model.predict(X_test)

In [ ]:
f1 = metrics.f1_score(y_test, y_test_pred, average='weighted')
f1

1.0

### 4. Оптимизируйте гиперпараметры, используя следующие наборы гиперпараметров: param_grid = [{'solver': ['lbfgs', 'newton-cg'],'penalty': ['l2']}, {'C': np.logspace(0,4,10), 'penalty': ['l2', 'l1'], 'solver': ['liblinear']}, {'C': np.logspace(0,4,10), 'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']},] по метрике accuracy. Реализуйте поиск лучших гиперпараметров с перекрёстной проверкой, используя класс GridSearchCV. Выведете результаты поиска в виде датафрейма, состоящего из двух колонок: в первой колонке все перебираемые сочитания гиперпараметров, а во второй соответствующие им средние значения метрики accuracy. Выведите значения лучших гиперпараметров.

In [ ]:
param_grid = [
    {'solver': ['lbfgs', 'newton-cg'], 'penalty': ['l2']},
    {'C': np.logspace(0, 4, 10), 'penalty': ['l2', 'l1'], 'solver': ['liblinear']},
    {'C': np.logspace(0, 4, 10), 'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']},
]

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5)

In [ ]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
             param_grid=[{'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']},
                         {'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                          'penalty': ['l2', 'l1'], 'solver': ['liblinear']},
                         {'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                          'penalty': ['l2'],
                          'solver': ['lbfgs', 'newton-cg']}],
             scoring='accuracy')

In [ ]:
results = pd.DataFrame(grid_search.cv_results_)[['params', 'mean_test_score']]

In [ ]:
results

,params,mean_test_score
0,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.960615
1,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.960615
2,"{'C': 1.0, 'penalty': 'l2', 'solver': 'libline...",0.937538
3,"{'C': 1.0, 'penalty': 'l1', 'solver': 'libline...",0.921538
4,"{'C': 2.7825594022071245, 'penalty': 'l2', 'so...",0.937231
5,"{'C': 2.7825594022071245, 'penalty': 'l1', 'so...",0.945231
6,"{'C': 7.742636826811269, 'penalty': 'l2', 'sol...",0.952923
7,"{'C': 7.742636826811269, 'penalty': 'l1', 'sol...",0.952923
8,"{'C': 21.544346900318832, 'penalty': 'l2', 'so...",0.952923
9,"{'C': 21.544346900318832, 'penalty': 'l1', 'so...",0.968615


In [ ]:
grid_search.best_params_

{'C': 21.544346900318832, 'penalty': 'l1', 'solver': 'liblinear'}

### 5. Выведите значение метрики accuracy на тестовых данных. Cделайте предсказание для лучшей модели и выведите метрику f1-score. Сравните полученные метрики с метриками, полученными без оптимизации гиперпараметров.

In [ ]:
grid_search.score(X_test, y_test)

0.9565217391304348

In [ ]:
y_test_pred = grid_search.predict(X_test)
metrics.f1_score(y_test, y_test_pred, average='weighted')

0.9550514597773576

### 6. Осуществите рандомизированный решетчатый поиск лучших гиперпараметров по метрике accuracy для той же самой сетки гиперпараметров.

In [ ]:
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=10,
    scoring='accuracy',
    cv=5,
    random_state=0,
)

In [ ]:
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
                   param_distributions=[{'penalty': ['l2'],
                                         'solver': ['lbfgs', 'newton-cg']},
                                        {'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                                         'penalty': ['l2', 'l1'],
                                         'solver': ['liblinear']},
                                        {'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                                         'penalty': ['l2'],
                                         'solver': ['lbfgs', 'newton-cg']}],
                   random_state=0, scoring='accuracy')

In [ ]:
random_search.best_params_

{'solver': 'lbfgs', 'penalty': 'l2', 'C': 59.94842503189409}

In [ ]:
results = pd.DataFrame(random_search.cv_results_)[['params', 'mean_test_score']]
results

,params,mean_test_score
0,"{'solver': 'lbfgs', 'penalty': 'l2', 'C': 59.9...",0.968615
1,"{'solver': 'lbfgs', 'penalty': 'l2', 'C': 1291...",0.968615
2,"{'solver': 'newton-cg', 'penalty': 'l2', 'C': ...",0.952923
3,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...",0.937231
4,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...",0.952923
5,"{'solver': 'newton-cg', 'penalty': 'l2', 'C': ...",0.952923
6,"{'solver': 'lbfgs', 'penalty': 'l2', 'C': 21.5...",0.968615
7,"{'solver': 'liblinear', 'penalty': 'l1', 'C': ...",0.968615
8,"{'solver': 'newton-cg', 'penalty': 'l2', 'C': ...",0.968615
9,"{'solver': 'newton-cg', 'penalty': 'l2', 'C': ...",0.968615


### 7. Осуществите решетчатый поиск лучших гиперпараметров по метрике f1 для той же самой сетки гиперпараметров.

In [ ]:
param_grid = [
    {'solver': ['lbfgs', 'newton-cg'], 'penalty': ['l2']},
    {'C': np.logspace(0, 4, 10), 'penalty': ['l2', 'l1'], 'solver': ['liblinear']},
    {'C': np.logspace(0, 4, 10), 'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']},
]

In [ ]:
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=5,
)

In [ ]:
grid_search.fit(X_train, y_train)
grid_search.best_params_

{'C': 21.544346900318832, 'penalty': 'l1', 'solver': 'liblinear'}

In [ ]:
acc = grid_search.score(X_test, y_test)
acc

0.9550514597773576

In [ ]:
y_test_pred = grid_search.predict(X_test)
f1 = metrics.f1_score(y_test, y_test_pred, average='weighted')
f1

0.9550514597773576